In [4]:
from collections import namedtuple
import math
import functools
import numpy as np
import csv
import pprint
import random
import xml.etree.ElementTree as ET

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import collections as mc

We start off by shamelessly stealing the general structure of our solutiou out of the lab.

In [5]:
Vertex = namedtuple('Vertex', ['id', 'x', 'y', 'order'])

In [6]:
@functools.lru_cache(maxsize=None)
def distance(v1, v2):
    return math.sqrt((v1.x - v2.x)**2+(v1.y - v2.y)**2)

In [7]:
def fitness(vertices, distance, solution):
    solution_distance = 0
    for x, y in zip(solution, solution[1:]):
        solution_distance += distance(vertices[x], vertices[y])
    solution_distance += distance(vertices[solution[-1]], vertices[solution[0]])
    return solution_distance

In [8]:
def initialize_pheromone(N):
    return 0.01*np.ones(shape=(N,N))

def update_pheromone(pheromones_array, solutions, fits, Q=100, rho=0.6):
    pheromone_update = np.zeros(shape=pheromones_array.shape)
    for solution, fit in zip(solutions, fits):
        for x, y in zip(solution, solution[1:]):
            pheromone_update[x][y] += Q/fit
        pheromone_update[solution[-1]][solution[0]] += Q/fit
    
    return (1-rho)*pheromones_array + pheromone_update

In [9]:
def generate_solutions(vertices, pheromones, distance, N, alpha=1, beta=3):
    
    def compute_prob(v1, v2):
        dist = 1/distance(vertices[v1], vertices[v2])
        tau = pheromones[v1, v2]
        ret = pow(tau, alpha) * pow(dist,beta)
        return ret if ret > 0.000001 else 0.000001

    pheromones_shape = pheromones.shape[0]
    for i in range(N):
        available = list(range(pheromones_shape))
        solution = [np.random.randint(0, pheromones_shape)]
        available.remove(solution[0])
        while available:
            probs = np.array(list(map(lambda x: compute_prob(solution[-1], x), available)))
            selected = np.random.choice(available, p=probs/sum(probs)) # vyber hrany
            solution.append(selected)
            available.remove(selected)
        yield solution

In [10]:
def ant_solver(vertices, distance, ants=10, max_iterations=1000, alpha=1, beta=3, Q=100, rho=0.8):
    pheromones = initialize_pheromone(len(vertices))
    best_solution = None
    best_fitness = float('inf')
    
    for i in range(max_iterations):
        solutions = list(generate_solutions(vertices, pheromones, distance, ants, alpha=alpha, beta=beta))
        fits = list(map(lambda x: fitness(vertices, distance, x), solutions))
        pheromones = update_pheromone(pheromones, solutions, fits, Q=Q, rho=rho)
        
        for s, f in zip(solutions, fits):
            if f < best_fitness:
                best_fitness = f
                best_solution = s
        
        print(f'{i:4}, {np.min(fits):.4f}, {np.mean(fits):.4f}, {np.max(fits):.4f}')
    return best_solution, pheromones

In [ ]:
def read_xml_data(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()

    nodes = root.find('network').find('nodes')
    fleet = root.find('fleet')
    requests = root.find('requests')

    paired_requests = {}
    verticies = []
    depo = None

    for request in requests.findall('request'):
        node = int(request.get('node'))
        found = paired_requests.get(node, 0)
        paired_requests[node] = found+float(request.find('quantity').text)
    
    for node in nodes.findall('node'):
        type_of_node = int(node.get('type'))
        id = node.get('id')
        
        if type_of_node == 1:
            verticies.append(Vertex(id, float(node.find('cx').text), float(node.find('cy').text), paired_requests.get(id, 0)))
        else:
            depo = Vertex(id, float(node.find('cx').text), float(node.find('cy').text), 0)
    
    capacity = float(fleet.find('vehicle_profile').find('capacity').text)
    return verticies, depo, capacity, paired_requests

In [ ]:
def draw_result(vertices, pheromones, best_solution):
    lines = []
    colors = []
    for i, v1 in enumerate(vertices):
        for j, v2 in enumerate(vertices):
            lines.append([(v1.x, v1.y), (v2.x, v2.y)])
            colors.append(pheromones[i][j])

    lc = mc.LineCollection(lines, linewidths=np.array(colors))

    plt.figure(figsize=(12, 8))
    ax = plt.gca()
    ax.add_collection(lc)
    ax.autoscale()

    solution = best_solution

    # tady muzeme zkouset vliv jednotlivych parametru na vygenerovane reseni
    # solution = list(generate_solutions(vertices, pheromones, distance, N=1, alpha=3, beta=1))[0]

    print('Fitness: ', fitness(vertices, distance, solution))

    solution_vertices = [vertices[i] for i in solution]
    pprint.pprint(solution_vertices)

    solution_lines = []
    for i, j in zip(solution, solution[1:]):
        solution_lines.append([(vertices[i].x, vertices[i].y), (vertices[j].x, vertices[j].y)])
    solution_lines.append([(vertices[solution[-1]].x, vertices[solution[-1]].y), (vertices[solution[0]].x, vertices[solution[0]].y)])
    solutions_lc = mc.LineCollection(solution_lines, colors='red')
    ax.add_collection(solutions_lc)